In [2]:
import formats.futures_strategy_formats as fsf
import formats.strategy_followup_formats as sff
import formats.risk_pnl_formats as rpf
import my_sql_routines.futures_price_loader as fpl
import my_sql_routines.my_sql_utilities as msu
import get_price.presave_price as pp
import opportunity_constructs.futures_butterfly as fb
import contract_utilities.expiration as exp
import ta.prepare_daily as prep
import ta.email_reports as er

writer_out = sff.generate_futures_butterfly_followup_report(as_of_date=report_date, con=con)
writer_out = sff.generate_spread_carry_followup_report(as_of_date=report_date, con=con, writer=writer_out)
sff.generate_vcs_followup_report(as_of_date=report_date, con=con, writer=writer_out)
prep.move_from_dated_folder_2daily_folder(ext='ta', file_name='followup', folder_date=report_date)



In [1]:
import signals.option_signals as ops
ops.get_vcs_signals(ticker1='ESM2016',ticker2='ESU2016',settle_date=20160322)


{'current': {'imp_vol_ratio': 0.9148910991765964},
 'hist':       settleDates  TickerYear1  TickerMonth1  trDTE1  calDTE1  underlying1  \
 0        19940103         1994             6     116      164       467.05   
 1        19940104         1994             6     115      163       468.65   
 2        19940105         1994             6     114      162       469.20   
 3        19940106         1994             6     113      161       469.00   
 4        19940107         1994             6     112      160       472.35   
 5        19940110         1994             6     111      157       477.10   
 6        19940111         1994             6     110      156       476.45   
 7        19940112         1994             6     109      155       475.20   
 8        19940113         1994             6     108      154       474.60   
 9        19940114         1994             6     107      153       475.65   
 10       19940117         1994             6     106      150       474

In [4]:
folder_date = 20160616
fpl.update_futures_price_database_from_cme_file(con=con, settle_date=folder_date)

In [22]:
import opportunity_constructs.spread_carry as sc

#Carry Spread Bet Size: 864
spread_carry_output = sc.generate_spread_carry_sheet_4date(report_date=20160728)
spread_report = spread_carry_output['spread_report']
ticker_head_list = list(spread_report['tickerHead'].unique())
# Compare BO and S sizing

spread_report[spread_report['tickerHead']==ticker_head_list[19]]
#spread_report[spread_report['tickerHead']=='BO']

# there's a possible replacement for BO K-N

,ticker1,ticker2,tickerHead,front_tr_dte,carry,q_carry,reward_risk,price,q,upside,downside,change5,change10,change20
0,NGU2016,NGV2016,NG,22,NaN,NaN,NaN,-0.036,36,235.00,-182.15,NaN,NaN,NaN
1,NGV2016,NGX2016,NG,43,1140,-15,0.70,-0.150,51,454.40,-389.50,NaN,NaN,NaN
2,NGX2016,NGZ2016,NG,64,780,21,0.67,-0.228,30,342.15,-278.70,NaN,NaN,NaN
3,NGZ2016,NGF2017,NG,85,-1180,-6,-2.40,-0.110,36,117.15,-115.00,NaN,NaN,NaN
4,NGF2017,NGG2017,NG,106,-1270,-16,-4.50,0.017,52,67.15,-89.30,NaN,NaN,NaN
5,NGG2017,NGH2017,NG,126,-390,15,-0.93,0.056,37,105.00,-142.15,NaN,NaN,NaN
6,NGH2017,NGJ2017,NG,145,-2230,-35,-1.46,0.279,72,400.75,-645.80,NaN,NaN,NaN
7,NGJ2017,NGK2017,NG,168,2490,18,4.71,0.030,54,108.60,-115.00,NaN,NaN,NaN
8,NGK2017,NGM2017,NG,187,620,32,2.26,-0.032,22,70.00,-72.15,NaN,NaN,NaN
9,NGM2017,NGN2017,NG,209,-30,-21,-0.13,-0.029,43,52.15,-57.15,NaN,NaN,NaN


In [31]:
import signals.intraday_futures_signals as ifs
import ta.trade_fill_loader as tfl
import shared.statistics as stats

ticker_list = ['CZ2016','WZ2016']
#ticker_list = ['RBZ2016','CLZ2016']
#ticker_list = ['HOZ2016','CLZ2016']
#ticker_list = ['SX2016','BOZ2016','SMZ2016']
#ticker_list = ['CLZ2016','BZ2016']
#ticker_list = ['CLK2017','BJ2017']
ticker_list = ['SF2017','BOF2017','SMF2017']
#ticker_list = ['CLZ2016','BZ2016']

num_days_back_4intraday = 5
signals_output = ifs.get_intraday_spread_signals(ticker_list=ticker_list,date_to=20160725,
                                                 num_days_back_4intraday=num_days_back_4intraday)
print(signals_output['z'])
print(signals_output['recent_trend'])

#stats.get_regression_results({'y':aligned_data['spread_pnl1'],'x':aligned_data['spread_pnl_1']})

0.852780653062
-39.0633608815


In [10]:
import matplotlib.pyplot as plt
intraday_data = signals_output['intraday_data']
plt.plot(intraday_data['spread'])
plt.grid()         
         
plt.show()

In [32]:
intraday_data = signals_output['intraday_data']
print(intraday_data['spread'].mean())
print(intraday_data['spread'].std())
print(intraday_data['spread'].mean()+intraday_data['spread'].std())
print(intraday_data['spread'].mean()-intraday_data['spread'].std())

# -90 buy back

intraday_data.index[-1]

-93.04544536027697
1.906053279282396
-91.13939208099458
-94.95149863955936


Timestamp('2016-07-25 16:47:00')

In [12]:
import matplotlib.pyplot as plt
plt.plot(wuhu.index,wuhu['mid_p'])
plt.grid()         
         
plt.show()

In [23]:

import formats.futures_strategy_formats as fsf
fsf.generate_ifs_formatted_output()
        



In [14]:
pnl_frame

import pandas as pd


frame_out = pd.DataFrame({'ticker': ticker_frame, 'date':date_frame,'pnl': pnl_frame})
frame_out.sort('date',inplace=True)
frame_out['pnl'].cumsum()

import matplotlib.pyplot as plt
plt.plot(frame_out['pnl'].cumsum())
plt.grid()         
         
plt.show()

In [12]:
sum(frame_out['pnl']<0)

47

In [4]:
import signals.intraday_futures_signals as ifs
ifs.get_intraday_trend_signals(ticker='LCV2016',date_to=20160707)

LCV2016
20160707


{'daily_pnl': -260.00000000000227,
 'intraday_data':                   index         c1                                   \
                         best_bid_p best_bid_q best_ask_p best_ask_q   
 0   2016-07-07 08:31:00    112.950          1    112.950          2   
 1   2016-07-07 08:32:00    112.950          1    112.950          2   
 2   2016-07-07 08:33:00    112.675          4    112.750          5   
 3   2016-07-07 08:34:00    112.450          3    112.525          2   
 4   2016-07-07 08:35:00    112.450          1    112.475          1   
 5   2016-07-07 08:36:00    112.425          2    112.475          1   
 6   2016-07-07 08:37:00    112.325          5    112.400          5   
 7   2016-07-07 08:38:00    112.350          1    112.400          1   
 8   2016-07-07 08:39:00    112.525          2    112.600         16   
 9   2016-07-07 08:40:00    112.625          2    112.650          1   
 10  2016-07-07 08:41:00    112.775          1    112.825          4   
 11  2016-07

In [9]:


import signals.intraday_futures_signals as ifs
signals_output = ifs.get_intraday_trend_signals(ticker='LCV2016',date_to=20160630)
intraday_data = signals_output['intraday_data']

import matplotlib.pyplot as plt
plt.plot(range(len(intraday_data.index)),intraday_data['mid_p'],
         range(len(intraday_data.index)),intraday_data['ewma25'],
        range(len(intraday_data.index)),intraday_data['ewma100']
        )
plt.grid()         
         
plt.show()

LCV2016
20160630


In [13]:
import matplotlib.pyplot as plt
plt.plot(range(len(aligned_data.index)),aligned_data['c1']['close_price'],
        range(len(aligned_data.index)),aligned_data['c2']['close_price'])
plt.grid()         
         
plt.show()

-85.82285102420849
4.199943026208985
-81.6229079979995
-90.02279405041747


Timestamp('2016-07-14 14:32:00')

In [2]:
import matplotlib.pyplot as plt
plt.scatter(intraday_data['c1']['mid_p'],intraday_data['c2']['mid_p'])
plt.grid()         
         
plt.show()

In [5]:
import shared.statistics as stats

#yield_regress_output = stats.get_regression_results({'x':yield2, 'y':yield1,'x_current': yield2_current, 'y_current': yield1_current,
#                                                         'clean_num_obs': max(100, round(3*len(yield1.values)/4))})

stats.get_regression_results({'y':intraday_data['c1']['mid_p'].values,'x':intraday_data['c2']['mid_p'].values})

{'alpha': 1.1850901654380999,
 'beta': 0.98643764580320514,
 'conf_int': array([[ 1.10675436,  1.26342598],
        [ 0.98488015,  0.98799514]]),
 'residualstd': 0.043767005836800189,
 'rsquared': 99.838955405787345,
 'zscore': -0.77307191646706896}

In [5]:
import matplotlib.pyplot as plt
plt.plot(aligned_data['spread'])
plt.grid()         
         
plt.show()

In [4]:
import opportunity_constructs.utilities as opUtil
import contract_utilities.contract_meta_info as cmi
ticker_list = ['CLZ2016','HOZ2016']
amcb_output = [opUtil.get_aggregation_method_contracts_back(cmi.get_contract_specs(x)) for x in ticker_list]

[x['aggregation_method'] for x in amcb_output]

[1, 12]

In [1]:
import contract_utilities.expiration as exp
exp.doubledate_shift_bus_days(double_date=20160629,shift_in_days=4)

20160623

In [8]:
import matplotlib.pyplot as plt
plt.scatter(aligned_data['c1']['close_price'],aligned_data['c2']['close_price'])
plt.grid()         
         
plt.show()

In [41]:
import opportunity_constructs.utilities as oput
aligned_output = oput.get_aligned_futures_data(contract_list=['CZ2016','WZ2016'],
                              aggregation_method=12,
                             contracts_back=10,
                             date_to=20160628)

aligned_data = aligned_output['aligned_data']
p1 = aligned_data['c1']['close_price']
p2 = aligned_data['c2']['close_price']


import matplotlib.pyplot as plt
plt.plot(p1/p2)
plt.grid()         
         
plt.show()


In [45]:
import contract_utilities.expiration as exp
exp.get_days2_expiration(ticker='CLZ2017',date_to=20160629,instrument='futures')

{'cal_dte': 509,
 'expiration_datetime': datetime.date(2017, 11, 20),
 'tr_dte': 352}

In [21]:


import opportunity_constructs.intraday_future_spreads as ifs
import opportunity_constructs.utilities as opput
#wuhu = ifs.get_spreads_4date(date_to=20160628)

indx = 4
contract1 = wuhu['contract1'].iloc[indx]
contract2 = wuhu['contract2'].iloc[indx]

contract1

aligned_output = opput.get_aligned_futures_data_intraday(contract_list = [contract1,contract2],
                                       date_list=[20160623,20160624,20160627,20160628])

mid_1 = (aligned_output['c1']['best_bid_p']+aligned_output['c1']['best_ask_p'])/2
mid_2 = (aligned_output['c2']['best_bid_p']+aligned_output['c2']['best_ask_p'])/2

resid = 0.42*mid_1-mid_2


In [23]:
import matplotlib.pyplot as plt
plt.plot(resid)
plt.grid()         
         
plt.show()

In [24]:
import matplotlib.pyplot as plt
plt.scatter(mid_2,resid)
plt.grid()         
         
plt.show()

In [6]:
print(resid.mean())
print(resid.std())

-1483.3707493061981
6.542279100352497


In [5]:
import opportunity_constructs.utilities as opu
aligned_frame = opu.get_aligned_futures_data_intraday(contract_list=['CLZ2016','HOZ2016'],date_list=[20160622,20160623,20160624])

mid1 = (aligned_frame['c1']['best_bid_p']+aligned_frame['c1']['best_ask_p'])/2
mid2 = (aligned_frame['c2']['best_bid_p']+aligned_frame['c2']['best_ask_p'])/2

resid = mid1-0.42*mid2

import matplotlib.pyplot as plt
plt.plot(resid)
plt.grid()         
         
plt.show()

In [54]:
import matplotlib.pyplot as plt
plt.scatter(mid_p,mid_p2)
plt.grid()         
         
plt.show()

In [86]:
import opportunity_constructs.utilities as ocu
data_out = ocu.get_aligned_futures_data(contract_list=['CLZ2016','HOZ2016'],aggregation_method=12,contracts_back=2,date_to=20160622)
aligned_data = data_out['aligned_data']

In [92]:
aligned_data['c1']['close_price']

import matplotlib.pyplot as plt
import shared.statistics as stats
#plt.scatter(aligned_data['c1']['close_price'],aligned_data['c2']['close_price'])
#plt.grid()         
         
#plt.show()

regress_output = stats.get_regression_results({'y':aligned_data['c1']['close_price'],'x':aligned_data['c2']['close_price']})

resid = aligned_data['c1']['close_price']-42*aligned_data['c2']['close_price']


import matplotlib.pyplot as plt
plt.plot(resid)
plt.grid()         
         
plt.show()

In [78]:
plt.scatter(aligned_data['c1']['close_price'],aligned_data['c2']['close_price'])
plt.grid()         
         
plt.show()

In [81]:
regress_output['beta']

28.411566512349935

In [88]:
import matplotlib.pyplot as plt
plt.plot(aligned_data['c1']['close_price'])
plt.grid()         
         
plt.show()